# KAKENの配分区分マスタをローカルのMariaDBに保存するプログラム

### 事前準備
- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、configparserで ./config.ini として保存しておく。

### ここから本編
bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [2]:
# encoding: utf-8
from lxml import etree
import pandas as pd

In [3]:
tree = etree.parse('grants_masterxml_kaken/section_master_kakenhi.xml')

sectionlist = []
for section_table in tree.iterfind("section_table"):
    for section in section_table.iterfind("section"):
        section_name = section.find("name[@lang='ja']").text
        section_niicode = section.find("code[@type='nii']").text

        row = [
            section_niicode,
            section_name,
        ]
        
        sectionlist.append(row)
        
df = pd.DataFrame(sectionlist)
df.columns = ['section_niicode', 'section_name']
df

,section_niicode,section_name
0,1,本調査
1,2,成果とりまとめ
2,3,総括
3,4,学術定期刊行物
4,5,学術図書
5,6,二次刊行物
6,4,学術定期刊行物
7,5,学術図書
8,6,二次刊行物
9,7,現地調査


重複のデータがあるかどうか

In [4]:
df.duplicated().any()

True

重複データがあった。dropしておく。

In [5]:
df = df.drop_duplicates()
df.duplicated().any()

False

In [6]:
df.section_niicode.value_counts()

13    1
12    1
17    1
10    1
23    1
19    1
4     1
16    1
1     1
25    1
21    1
20    1
9     1
22    1
18    1
8     1
5     1
14    1
3     1
11    1
26    1
24    1
2     1
6     1
15    1
7     1
Name: section_niicode, dtype: int64

niicodeがユニークなので、インデックスに設定する

In [7]:
df = df.set_index('section_niicode')
df

,section_name
section_niicode,
1,本調査
2,成果とりまとめ
3,総括
4,学術定期刊行物
5,学術図書
6,二次刊行物
7,現地調査
8,調査総括
9,がん特別調査


ローカルのmariaDBに関する設定ファイルを読み込み（config.iniはgitに上げていないが、事前準備で作られているはず）

In [8]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

データベースにテーブル構造とデータの中身を書き込む

In [10]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('kaken_master_section', engine, if_exists='replace',
          dtype={
              'section_niicode': Integer,
              'section_name': String(256),
                })

2018-05-09 12:00:57,621 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-05-09 12:00:57,621 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:57,623 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-05-09 12:00:57,623 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:57,624 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-05-09 12:00:57,625 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:57,627 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-05-09 12:00:57,627 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:57,628 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-05-09 12:00:57,628 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 12:00:57,629 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-05-09 12

### おしまい
データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。